In [2]:
from sympy import sin, cos, symbols, lambdify
import numpy as np
x = symbols('x')
expr = sin(x) + cos(x)
expr
f = lambdify(x, expr, 'numpy')
a = np.array([1, 2])
f(a)
# type(f(a))

array([1.38177329, 0.49315059])

In [15]:
from gymnax.environments import environment, spaces
import numpy as np
import jax
import chex
action_space = spaces.Box(-1.0,1.0,shape=(2,3),dtype=np.float32)
rng = jax.random.PRNGKey(0)
action_space.sample(rng=rng)


TypeError: Cannot instantiate typing.Union

In [7]:

from quadjax.dynamics.dataclass import Action
import numpy as np
thrust = 1.0
tau = 0.5
env_action = Action(thrust=np.array([thrust, thrust]), tau = np.array([tau, tau]))
env_action


Action(thrust=array([1., 1.]), tau=array([0.5, 0.5]))

In [ ]:
import sympy as sp
from sympy.physics.mechanics import (
    dynamicsymbols,
    ReferenceFrame,
    Point,
    Particle,
    RigidBody,
    inertia,
)

# Define symbols
t = sp.Symbol("t")  # time
m = sp.Symbol("m", positive=True)  # mass of the quadrotor
I = sp.Symbol("I", positive=True)  # moment of inertia
g = sp.Symbol("g", positive=True)  # gravitational acceleration
l = sp.Symbol("l", positive=True)  # length of the rod
# mass of the object attached to the rod
mo = sp.Symbol("mo", positive=True)
thrust = sp.Function("thrust")(t)  # thrust force
tau = sp.Function("tau")(t)  # torque
f_rope = sp.Symbol("f_rope")  # force in the rope
# y displacement of the hook from the quadrotor center
delta_yh = sp.Symbol("delta_yh")
# z displacement of the hook from the quadrotor center
delta_zh = sp.Symbol("delta_zh")
params = [m, I, g, l, mo, delta_yh, delta_zh]
action = [thrust, tau]


y, z, theta, phi = dynamicsymbols("y z theta phi")
y_dot, z_dot, theta_dot, phi_dot = sp.diff(y, t), sp.diff(
    z, t), sp.diff(theta, t), sp.diff(phi, t)
y_ddot, z_ddot, theta_ddot, phi_ddot = sp.diff(y_dot, t), sp.diff(
    z_dot, t), sp.diff(theta_dot, t), sp.diff(phi_dot, t)
y_dot_val, z_dot_val, theta_dot_val, phi_dot_val = sp.symbols(
    "y_dot_val z_dot_val theta_dot_val phi_dot_val")
y_ddot_val, z_ddot_val, theta_ddot_val, phi_ddot_val = sp.symbols(
    "y_ddot_val z_ddot_val theta_ddot_val phi_ddot_val")
states = [y, z, theta, phi, y_dot, z_dot, theta_dot, phi_dot]
states_val = [y, z, theta, phi, y_dot_val,
                z_dot_val, theta_dot_val, phi_dot_val]
states_dot = [y_ddot, z_ddot, theta_ddot, phi_ddot, f_rope]
states_dot_val = [y_ddot_val, z_ddot_val,
                    theta_ddot_val, phi_ddot_val, f_rope]

states_val = [y, z, theta, phi, y_dot_val,
                  z_dot_val, theta_dot_val, phi_dot_val]

# Solve for the acceleration
A_taut_dyn = sp.zeros(5, 5)
b_taut_dyn = sp.zeros(5, 1)

# lambda A_taut_dyn
A_taut_dyn_func = sp.lambdify(
    params + states_val + action, A_taut_dyn, "jax")
b_taut_dyn_func = sp.lambdify(
    params + states_val + action, b_taut_dyn, "jax")